# More Examples

报告人：谷晓松

报告时间：2020/05/29

[在线观看幻灯片](https://github.com/hengxin/sat-smt-satisfying//blob/master/seminar/2020-05-29-Gu/more_examples.ipynb#/)

In [ ]:
!pip3 install RISE

## 内容预览

- 整数溢出的检测
- 正则表达式的匹配
- 格雷码

### 参考资料
- SAT SMT by Examples
- Z3Py Guide (Jupyter)

### 使用Z3验证代码加法是否会超出32位机器数范围

带符号32位加法会溢出吗？

![image.png](figs/ALU.png)

In [ ]:
from z3 import *
def func(a,b):
    return a+b

a32 , b32 , out32 = BitVecs('a32 b32 out32 ', 32)
out32_extended = BitVec('out32_extended ', 64)
a64 , b64 , out64 = BitVecs('a64 b64 out64 ', 64)
s=Optimize ()
s.add(out32 == func(a32 , b32))
s.add(out64 == func(a64 , b64))
s.add(a64== SignExt (32, a32))
s.add(b64== SignExt (32, b32))
s.add(out32_extended == SignExt (32, out32))
s.add(out64 != out32_extended)
s.minimize(a32)
s.minimize(b32)
if s.check ()== unsat:
    print "unsat: everything is OK"
    exit (0)
m=s.model ()

In [ ]:
a32=0x1 or 1
b32=0 x7fffffff or 2147483647
out32 =0 x80000000 or -2147483648
out32_extended =0 xffffffff80000000 or -2147483648
a64=0x1 or 1
b64=0 x7fffffff or 2147483647
out64 =0 x80000000 or 2147483648

另外还有算数平均数，绝对值函数的溢出分析

### 使用SMT求解所有匹配正则表达式的字符串
- 将RE转化为DFA
    + http://hokein.github.io/Automata.js
- 使用SMT求解匹配DFA的所有字符串

正则表达式：
$
(dark|light)?(red|blue|green)(ish)?
$

DFA:
![](figs/DFA.PNG)

定义状态转换函数

In [ ]:
#!/usr/bin/env python
from MK85 import *
BIT_WIDTH =16
INVALID_STATE =999
ACCEPTING_STATES =[13, 19, 23, 24]

# st - state
# i - input character
def transition (s, st , i):
    return s.If(And(st==0, i==ord('r')), s.BitVecConst (3, BIT_WIDTH),
    s.If(And(st==0, i==ord('b')), s.BitVecConst (4, BIT_WIDTH),
    s.If(And(st==0, i==ord('g')), s.BitVecConst (5, BIT_WIDTH),
    s.If(And(st==0, i==ord('d')), s.BitVecConst (1, BIT_WIDTH),
    s.If(And(st==0, i==ord('l')), s.BitVecConst (2, BIT_WIDTH),
    s.If(And(st==1, i==ord('a')), s.BitVecConst (6, BIT_WIDTH),
    s.If(And(st==2, i==ord('i')), s.BitVecConst (7, BIT_WIDTH),
    s.If(And(st==3, i==ord('e')), s.BitVecConst (8, BIT_WIDTH),
    s.If(And(st==4, i==ord('l')), s.BitVecConst (9, BIT_WIDTH),
    s.If(And(st==5, i==ord('r')), s.BitVecConst (10, BIT_WIDTH),
    s.If(And(st==6, i==ord('r')), s.BitVecConst (11, BIT_WIDTH),
    s.If(And(st==7, i==ord('g')), s.BitVecConst (12, BIT_WIDTH),
    s.If(And(st==8, i==ord('d')), s.BitVecConst (13, BIT_WIDTH),
    s.If(And(st==9, i==ord('u')), s.BitVecConst (14, BIT_WIDTH),
    s.If(And(st==10, i==ord('e')), s.BitVecConst (15, BIT_WIDTH),
    s.If(And(st==11, i==ord('k')), s.BitVecConst (16, BIT_WIDTH),
    s.If(And(st==12, i==ord('h')), s.BitVecConst (17, BIT_WIDTH),
    s.If(And(st==13, i==ord('i')), s.BitVecConst (18, BIT_WIDTH),
    s.If(And(st==14, i==ord('e')), s.BitVecConst (19, BIT_WIDTH),
    s.If(And(st==15, i==ord('e')), s.BitVecConst (20, BIT_WIDTH),
    s.If(And(st==16, i==ord('r')), s.BitVecConst (3, BIT_WIDTH),
    s.If(And(st==16, i==ord('b')), s.BitVecConst (4, BIT_WIDTH),
    s.If(And(st==16, i==ord('g')), s.BitVecConst (5, BIT_WIDTH),
    s.If(And(st==17, i==ord('t')), s.BitVecConst (21, BIT_WIDTH),
    s.BitVecConst(INVALID_STATE , 16)))))))))))))))))))))))))))))))))

求解所有符合DFA的字符串

In [ ]:
def make_FSM(length):
    s=MK85(verbose =0)
    states =[s.BitVec('states_%d' % i,BIT_WIDTH) for i in range(length)]
    inputs =[s.BitVec('inputs_%d' % i,BIT_WIDTH) for i in range(length -1)]

    # initial state:
    s.add(states [0]==0)

    # the last state must be equal to one of the acceping states
    s.add(Or(*[ states[length -1]==i for i in ACCEPTING_STATES ]))

    # all states are in limits ...
    for i in range(length):
        s.add(And(states[i]>=0, states[i] <=24))
    s.add(states[i]!= INVALID_STATE)

    # "insert" transition () functions between subsequent states
    for i in range(length -1):
        s.add(states[i+1] == transition(s, states[i], inputs[i]))

In [ ]:
    # enumerate results:
    results =[]
    while s.check ():
        m=s.model ()
        #print m
        print_model(m, length , inputs)
        # add the current solution negated:
        tmp =[]
        for pair in m:
            tmp.append(s.var_by_name(pair) == m[pair])
        s.add(expr.Not(And(*tmp)))

for l in range (2 ,15):
    make_FSM(l)

In [ ]:
red
blue
green
redish
darkred
blueish
darkblue
greenish
lightred
lightblue
darkgreen
lightgreen
darkredish
darkblueish
lightredish
darkgreenish
lightblueish
lightgreenish

检测某个字符串是否匹配给定的RE:方法类似，检测最终是否在终止状态

### 格雷码

![](figs/encoder.PNG)

普通格雷码

![](figs/gray.PNG)

平衡格雷码？
- 每一位改变的次数尽可能相同

In [3]:
from z3 import *
BITS=4

# how many times a run of bits for each bit can be changed (max).
# it can be 4 for 4-bit Gray code or 8 for 5-bit code.
# 12 for 6-bit code (maybe even less)
CHANGES_MAX =4

ROWS =2** BITS
MASK=ROWS -1 # 0x1f for 5 bits , 0xf for 4 bits , etc

def bool_to_int (b):
    if b==True:
        return 1
    return 0

s=Solver ()

# add a constraint: Hamming distance between two bitvectors must be 1
# i.e., two bitvectors can differ in only one bit.
# for 4 bits it works like that:
# s.add(Or(
# And(a3!=b3 ,a2==b2 ,a1==b1 ,a0==b0),
# And(a3==b3 ,a2!=b2 ,a1==b1 ,a0==b0),
# And(a3==b3 ,a2==b2 ,a1!=b1 ,a0==b0),
# And(a3==b3 ,a2==b2 ,a1==b1 ,a0!=b0)))
def hamming1(l1 , l2):
    assert len(l1)==len(l2)
    r=[]
    for i in range(len(l1)):
        t=[]
        for j in range(len(l1)):
            if i==j:
                t.append(l1[j]!=l2[j])
            else:
                t .append(l1[j]==l2[j])
        r.append(And(t))
    s.add(Or(r))

# add a constraint: bitvectors must be different.
# for 4 bits works like this:
# s.add(Or(a3!=b3 , a2!=b2 , a1!=b1 , a0!=b0))
def not_eq(l1 , l2):
    assert len(l1)==len(l2)
    t=[l1[i]!=l2[i] for i in range(len(l1))]
    s.add(Or(t))

code =[[ Bool('code_%d_%d' % (r,c)) for c in range(BITS)] for r in range(ROWS)]
ch=[[ Bool('ch_%d_%d' % (r,c)) for c in range(BITS)] for r in range(ROWS)]

# each rows must be different from a previous one and a next one by 1 bit:
for i in range(ROWS):
    # get bits of the current row:
    lst1=[code[i][bit] for bit in range(BITS)]
    # get bits of the next row.
    lst2=[code[(i+1)&MASK][bit] for bit in range(BITS)]
    hamming1(lst1 , lst2)

# no row must be equal to any another row:
for i in range(ROWS):
    for j in range(ROWS):
        if i==j:
            continue
        lst1 =[ code[i][bit] for bit in range(BITS)]
        lst2 =[ code[j][bit] for bit in range(BITS)]
        not_eq(lst1 , lst2)

# 1 in ch[] table means that run of 1's has been changed to run of 0's, or back.
# "run" change detected using simple XOR:
for i in range(ROWS):
    for bit in range(BITS):
        s.add(ch[i][bit ]== Xor(code[i][bit],code[(i+1)&MASK][bit]))

# only CHANGES_MAX of 1 bits is allowed in ch[] table for each bit:
for bit in range(BITS):
    t=[ch[i][bit] for i in range(ROWS)]
    # AtMost () takes arguments like:
    # AtMost(v1 , v2 , v3 , v4 , 2) <- this means , only 2 booleans (or less) from the list
    # can be True.
    s.add(AtMost (*(t+[ CHANGES_MAX ])))

result=s.check ()
if result == unsat:
    print("unsat!")
    exit (0)
m=s.model ()

# get the model.
print ("code table:")

for i in range(ROWS):
    t=""
    for bit in range(BITS):
        # comma at the end means "no newline ":
        t=t+str(bool_to_int(is_true(m[code[i][BITS -1-bit ]])))+" "
    print (t)

print ("ch table:")
stat ={}
for i in range(ROWS):
    t=""
    for bit in range(BITS):
        x=is_true(m[ch[i][BITS -1-bit ]])
        if x:
            stat[bit]= stat.get(bit , 0)+1
        # comma at the end means "no newline ":
        t=t+str(bool_to_int(x))+" "
    print (t)
print ("stat (bit number: number of changes): ", stat)



code table:
1 0 0 1 
0 0 0 1 
0 0 1 1 
0 0 1 0 
0 0 0 0 
0 1 0 0 
0 1 0 1 
1 1 0 1 
1 1 0 0 
1 0 0 0 
1 0 1 0 
1 1 1 0 
0 1 1 0 
0 1 1 1 
1 1 1 1 
1 0 1 1 
ch table:
1 0 0 0 
0 0 1 0 
0 0 0 1 
0 0 1 0 
0 1 0 0 
0 0 0 1 
1 0 0 0 
0 0 0 1 
0 1 0 0 
0 0 1 0 
0 1 0 0 
1 0 0 0 
0 0 0 1 
1 0 0 0 
0 1 0 0 
0 0 1 0 
stat (bit number: number of changes):  {0: 4, 2: 4, 3: 4, 1: 4}
